In [ ]:
#import import library
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#changing the working directory
%cd /content/gdrive/My Drive/LOMBA

/content/gdrive/My Drive/LOMBA


In [ ]:
#import dataset
df = pd.read_csv('dataset_poi.csv')
df = df.drop(columns="Unnamed: 0")
df.head()

,Sentence,Word,Tag
0,1,jl,O
1,1,kapuk,O
2,1,timur,O
3,1,delta,O
4,1,sili,O


In [ ]:
#Info dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2360706 entries, 0 to 2360705
Data columns (total 3 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   Sentence  int64 
 1   Word      object
 2   Tag       object
dtypes: int64(1), object(2)
memory usage: 54.0+ MB


In [ ]:
class SentenceGetter(object):
    
    def __init__(self, dataset):
        self.n_sent = 1
        self.dataset = dataset
        self.empty = False
        agg_func = lambda s: [(w, t) for w,t in zip(s["Word"].values.tolist(),
                                                        s["Tag"].values.tolist())]
        self.grouped = self.dataset.groupby("Sentence").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [ ]:
getter = SentenceGetter(df)
sentences = getter.sentences

In [ ]:
#mencari kalimat terpanjang dalam dataset
maxlen = max([len(s) for s in sentences])
print ('Maximum sequence length:', maxlen)

Maximum sequence length: 36


In [ ]:
df_dict = pd.read_csv('df_new_dict.csv')
df_dict = df_dict.drop(columns="Unnamed: 0")
df_dict.iloc[0,0]=str(df_dict.iloc[0,0])

In [ ]:
words = df_dict['Words'].values

In [ ]:
words[87406]

'ENDPAD'

In [ ]:
n_words = len(words); n_words

87407

In [ ]:
tags = list(set(df["Tag"].values))

In [ ]:
n_tags = len(tags); n_tags

3

In [ ]:
word_list = df_dict['Words'].values
key_list = df_dict['Key'].values
word2idx = dict(zip(word_list, key_list))

In [ ]:
tag2idx ={'B-POI': 1, 'I-POI': 0, 'O': 2}

In [ ]:
tags[2]

'O'

In [ ]:
from keras.preprocessing.sequence import pad_sequences
X = [[word2idx[w[0]] for w in s] for s in sentences]
print(X[0])

[50845, 15652, 24998, 44692, 53189, 47101, 64955, 26060, 1310, 53476, 17121, 62043, 61479]


In [ ]:
X = pad_sequences(maxlen=36, sequences=X, padding="post",value=87406)
print(X[0])

[50845 15652 24998 44692 53189 47101 64955 26060  1310 53476 17121 62043
 61479 87406 87406 87406 87406 87406 87406 87406 87406 87406 87406 87406
 87406 87406 87406 87406 87406 87406 87406 87406 87406 87406 87406 87406]


In [ ]:
y = [[tag2idx[w[1]] for w in s] for s in sentences]
print(y[5])

[2, 2, 2, 2, 2, 1, 0, 0]


In [ ]:
y = pad_sequences(maxlen=36, sequences=y, padding="post", value=tag2idx["O"])
print(y)

[[2 2 2 ... 2 2 2]
 [2 2 2 ... 2 2 2]
 [2 2 2 ... 2 2 2]
 ...
 [2 2 2 ... 2 2 2]
 [2 2 2 ... 2 2 2]
 [1 0 0 ... 2 2 2]]


In [ ]:
from keras.utils import to_categorical
y = [to_categorical(i, num_classes=n_tags) for i in y]
print(y[0])

[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
mc = ModelCheckpoint('best_model_poi.h5', monitor='val_loss', mode='min', save_best_only=True)

In [ ]:
from keras.optimizers import Adam
opt = Adam(learning_rate=0.01)

In [ ]:
Y = np.array(y)

In [ ]:
from sklearn.model_selection import KFold
seed = 7
np.random.seed(seed)
kfold = KFold(n_splits=5, shuffle=True, random_state=seed)
cvscores = []
test_loss = 1
for train, test in kfold.split(X, Y):
  # create model
  input=Input(shape=(36,))
  model=Embedding(input_dim=n_words, output_dim=36, input_length=36)(input)
  model=Dropout(0.2)(model)
  model=Bidirectional(LSTM(units=117, return_sequences=True, activity_regularizer=l2(0.01), recurrent_dropout=0.2))(model)
  out=TimeDistributed(Dense(n_tags, activation="softmax"))(model)
  model=Model(input, out)
	# Compile model
  model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"])
	# Fit the model
  model.fit(X[train], Y[train], batch_size=32, epochs=5, validation_split=0.2, verbose=1, callbacks=[mc,es])
	# evaluate the model
  scores = model.evaluate(X[test], Y[test], verbose=1)
  if test_loss>scores[0]:
    test_loss=scores[0]
    model.save_weights('best_model_cv_final_poi.h5')
  print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
  cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

Epoch 1/5
6000/6000 [==============================] - 1124s 187ms/step - loss: 0.0858 - accuracy: 0.9791 - val_loss: 0.0391 - val_accuracy: 0.9845
Epoch 2/5
6000/6000 [==============================] - 1140s 190ms/step - loss: 0.0335 - accuracy: 0.9876 - val_loss: 0.0377 - val_accuracy: 0.9849
Epoch 3/5
6000/6000 [==============================] - 1232s 205ms/step - loss: 0.0271 - accuracy: 0.9903 - val_loss: 0.0386 - val_accuracy: 0.9849
Epoch 00003: early stopping
1875/1875 [==============================] - 46s 25ms/step - loss: 0.0393 - accuracy: 0.9846
accuracy: 98.46%
Epoch 1/5
6000/6000 [==============================] - 1275s 212ms/step - loss: 0.0769 - accuracy: 0.9782 - val_loss: 0.0402 - val_accuracy: 0.9839
Epoch 2/5
6000/6000 [==============================] - 1150s 192ms/step - loss: 0.0335 - accuracy: 0.9876 - val_loss: 0.0389 - val_accuracy: 0.9844
Epoch 3/5
6000/6000 [==============================] - 1148s 191ms/step - loss: 0.0276 - accuracy: 0.9900 - val_loss: 0.03

In [ ]:
input = Input(shape=(36,))
model = Embedding(input_dim=n_words, output_dim=36, input_length=36)(input)
model = Dropout(0.2)(model)
model = Bidirectional(LSTM(units=117, return_sequences=True, activity_regularizer = l2(0.01), recurrent_dropout=0.2))(model)
out = TimeDistributed(Dense(n_tags, activation="softmax"))(model)  # softmax output layer

In [ ]:
model = Model(input, out)

In [ ]:
model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 36)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 36, 36)            3146652   
_________________________________________________________________
dropout (Dropout)            (None, 36, 36)            0         
_________________________________________________________________
bidirectional (Bidirectional (None, 36, 234)           144144    
_________________________________________________________________
time_distributed (TimeDistri (None, 36, 3)             705       
Total params: 3,291,501
Trainable params: 3,291,501
Non-trainable params: 0
_________________________________________________________________


In [ ]:
print(words[87406])

ENDPAD


In [ ]:
history = model.fit(X_train, np.array(y_train), batch_size=32, epochs=3, validation_split=0.2, verbose=1, callbacks=[es,mc])

Epoch 1/3
6000/6000 [==============================] - 574s 95ms/step - loss: 0.1577 - accuracy: 0.9782 - val_loss: 0.0453 - val_accuracy: 0.9842
Epoch 2/3
6000/6000 [==============================] - 568s 95ms/step - loss: 0.0347 - accuracy: 0.9889 - val_loss: 0.0393 - val_accuracy: 0.9850
Epoch 3/3
6000/6000 [==============================] - 573s 96ms/step - loss: 0.0227 - accuracy: 0.9927 - val_loss: 0.0412 - val_accuracy: 0.9846
Epoch 00003: early stopping


In [ ]:
model.evaluate(X_test,np.array(y_test))

1875/1875 [==============================] - 32s 17ms/step - loss: 0.0405 - accuracy: 0.9848


[0.04049307107925415, 0.9847740530967712]

In [ ]:
model.load_weights('best_model_cv_final_poi.h5')

In [ ]:
from tqdm import tqdm

pred_ = []
true_ = []
id_test = range(len(X_test))
for i in tqdm(id_test):
  p = model.predict(np.array([X_test[i]]))
  p = np.argmax(p, axis=-1)
  p_trans = np.transpose(p)
  count = 0
  for x in p_trans :
    if x == tag2idx["I-POI"] :
      if p_trans[count-1] == tag2idx["O"] :
        p_trans[count-1] = tag2idx["B-POI"]
        count += 1
      else :
        count += 1
    else :
      count += 1
    p_fin = np.transpose(p_trans)
  q = np.array(y_test[i])
  q = np.argmax(q, axis=-1)
  pred_.append(p_fin[0])
  true_.append(q)

100%|██████████| 60000/60000 [51:08<00:00, 19.55it/s]


In [ ]:
true_temp = np.array(true_)
pred_temp = np.array(pred_)

In [ ]:
analyze = []
true_score=[]
dummy = 0
for i in tqdm(id_test):
  for j in range(0,36):
    if true_temp[i,j]==pred_temp[i,j]:
      dummy += 1
    else :
      analyze.append(0)
      dummy = 0
      break
  if dummy == 36 :
    analyze.append(1)
    true_score.append(1)
    dummy = 0
  else :
    dummy = 0


print("acc : ",len(true_score)/len(id_test)*100,"%")

100%|██████████| 60000/60000 [00:01<00:00, 50955.34it/s]

acc :  83.33166666666668 %


In [ ]:
ready_poi = []
for i in tqdm(id_test):
  ans = ''
  for w,pred in zip(X_test[i],pred_[i]):
    if tags[pred] == "B-POI":
      ans = ans + str(words[w])
    elif tags[pred] == "I-POI":
      ans = ans + ' ' + str(words[w])
    else:
      ans = ans
  ready_poi = np.append(ready_poi,ans)

In [ ]:
ready_poi[0:10]

array(['', 'lyga tas', '', '', '', '', '', '', '', ''], dtype='<U65')

RUN THIS AFTER PREDICTING X_TEST

In [ ]:
test = pd.read_csv("df_new.csv")
data_test = test.drop(['Unnamed: 0'],axis=1)

In [ ]:
data_test.tail(20)

,Sentence,Word
392687,49998,comma
392688,49998,nasio
392689,49998,comma
392690,49999,graha
392691,49999,indah
392692,49999,pamulang
392693,49999,jl
392694,49999,dot
392695,49999,mujair
392696,49999,raya


In [ ]:
class SentenceGetter(object):
    
    def __init__(self, dataset):
        self.n_sent = 1
        self.dataset = dataset
        self.empty = False
        agg_func = lambda s: [(w) for w in zip(s["Word"].values.tolist())]
        self.grouped = self.dataset.groupby("Sentence").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [ ]:
getter_test = SentenceGetter(data_test)
sentences_test = getter_test.sentences

In [ ]:
X_fintest = [[word2idx[w[0]] for w in s] for s in sentences_test]
X_fintest = pad_sequences(maxlen=36, sequences=X_fintest, padding="post",value=87406)
print(X_fintest[0])

[73351 16517 26887 24589 33464 30505 77919 43143 87406 87406 87406 87406
 87406 87406 87406 87406 87406 87406 87406 87406 87406 87406 87406 87406
 87406 87406 87406 87406 87406 87406 87406 87406 87406 87406 87406 87406]


In [ ]:
from tqdm import tqdm

pred_ = []
true_ = []
id_test = range(len(X_fintest))
for i in tqdm(id_test):
  p = model.predict(np.array([X_fintest[i]]))
  p = np.argmax(p, axis=-1)
  p_trans = np.transpose(p)
  count = 0
  for x in p_trans :
    if x == tag2idx["I-POI"] :
      if p_trans[count-1] == tag2idx["O"] :
        p_trans[count-1] = tag2idx["B-POI"]
        count += 1
      else :
        count += 1
    else :
      count += 1
    p_fin = np.transpose(p_trans)
  pred_.append(p_fin[0])

In [ ]:
pred_[6]

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [ ]:
print("{:14} ({:5}): {}".format("Word", "True", "Pred"))
for w,pred in zip(X_fintest[11],pred_[11]):
    print("{:14}: {}".format(words[w],tags[pred]))

Word           (True ): Pred
kemur         : O
vii           : O
comma         : O
no            : O
95            : O
glodok        : O
rt            : O
5             : O
rw            : O
5             : O
taman         : O
sari          : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O


In [ ]:
poi_pred_final=[]
for i in words:
  word_=str(i).replace(" dot ",". ")
  word_=word_.replace(" dot","")
  word_=word_.replace("dot ","")
  word_=word_.replace(" comma ",", ")
  word_=word_.replace("comma ","")
  word_=word_.replace(" comma","")
  poi_pred_final.append(word_)

In [ ]:
ready_poi = []
for i in tqdm(id_test):
  ans = ''
  for w,pred in zip(X_fintest[i],pred_[i]):
    if tags[pred] == "B-POI":
      ans = ans + str(poi_pred_final[w]) + ' '
    elif tags[pred] == "I-POI":
      ans = ans + str(poi_pred_final[w]) + ' '
    else:
      ans = ans
  if ans!="":
    ready_poi = np.append(ready_poi,ans[:-1])
  else:
    ready_poi = np.append(ready_poi,ans)

In [ ]:
ready_poi[-100:]

array(['', '', '', 'kedai kopi nusantar', 'ketua', '', '', '', '',
       'bumi moro', 'perumahan griya kencana', '', '', '', '',
       'kp dot tanah ungkuk', 'depot bu deon', 'bakso cak hasan madci',
       '', '', '', '', '', '', '', '', 'lapangan kopen', '', '', '',
       'perumahan tni al griya samudra asri', '', 'sd neg 022ii lempur',
       '', 'surya cell', '', 'krenchise fried', '', '', '', '', '',
       'ud untung', '', '', 'bursa sec komp', '', '', '',
       'pp dot raudatul ulum2 comma cume', 'apt kf pepito canggu', '', '',
       '', '', 'gedung ot', '', '', '', '', '', 'cpp merauke', 'an nur',
       '', '', '', 'optik qu', 'asrama mahasiswa its', '', 'king s gym',
       '', 'ratna mini mart', 'mi swasta kasih bunda', '', '', '', '', '',
       '', 'toko mona', '', '', 'warmindo taman ringin', '', '', '',
       'komplek mega com', 'toko', '', '', '', '', '', '', '',
       'toko mbak farid', 'tk dot ridho kids',
       'mart dan roti bakar malabar', 'graha indah pamu

In [ ]:
poi_pred_final=[]
for i in ready_poi:
  word_=i.replace(" dot ",". ")
  word_=word_.replace(" dot",". ")
  word_=word_.replace("dot ",". ")
  word_=word_.replace(" comma ",", ")
  word_=word_.replace("comma ","")
  word_=word_.replace(" comma","")
  poi_pred_final.append(word_)

In [ ]:
poi_df = pd.DataFrame(poi_pred_final, columns = ['POI'])

In [ ]:
poi_df.head(20)

,POI
0,
1,
2,asma laun
3,ud agung
4,
5,
6,
7,
8,tk pam
9,


##PERBAIKAN

In [ ]:
poi_df.to_csv('poi_test_1.csv')

In [ ]:
df_test = pd.read_csv('test.csv')

In [ ]:
false_alarm=[]
poi_word=[]
idx=[]
for i in range(len(poi_df)):
  if poi_df.iloc[i,0]!="":
    if poi_df.iloc[i,0] not in df_test.iloc[i,1]:
      idx.append(i)
      false_alarm.append(poi_df.iloc[i,0])
      poi_word.append(df_test.iloc[i,1])

In [ ]:
len(false_alarm)

53

In [ ]:
i = 17
print(false_alarm[i],"--",poi_word[i])

dipo tower dipo tower -- jl gatot subroto kv 50-52 dipo tower loby receptionist loby utama dipo tower


In [ ]:
!pip install fuzzywuzzy
!pip install python-Levenshtein

In [ ]:
from fuzzywuzzy import fuzz
#string1 raw string 2 poi/street
def find_keyword(string1,string2):
  keyword = ""
  string1 = string1.split()
  for i in string1:
    if i!='' and fuzz.ratio(i,string2) >= 70 and len(keyword.split())<len(string2.split()):
      keyword+=i+" "
  return(keyword[:-1])

In [ ]:
false_alarm[101:111]

['dusun iwan masjid darul hamidy',
 'wisma sepak bola eks',
 'pb bumi agung batu alam',
 'sds adv cira',
 'pijat tunanetra pak rohim nh. nad',
 'pt. mega prima persada',
 'mas al kautsar',
 'anugrah jaya motor warung muncang',
 'tk bunga tpu utan jati',
 'jakarta design center slipi petamburan']

In [ ]:
true_poi=[]
for i,j in zip(false_alarm,poi_word):
  start_i=i.split()[0]
  end_i=i.split()[-1]
  #print(end_i,"----")
  poi_w = j.split()
  start_con=False
  poi_w_=""
  for k in poi_w:
    if find_keyword(k,start_i)==k:
      start_con=True
    if start_con:
      poi_w_+=k+' '
    if find_keyword(k,end_i)==k:
      start_con=False
  true_poi.append(poi_w_[:-1])

In [ ]:
true_poi[0:10]

['warung kuning dekat basecamp gunung andong via',
 'sisun kel.,',
 'balilaris pie susu & lapis legit by',
 'karang sari kpri makmur',
 'masjid al-muttaqin',
 'konter data cell ) komplek puri citra',
 'pizza & burger,',
 "kiv baby 'n kids shop,",
 '',
 'smps plus as-sa adah']

In [ ]:
idx[0:10]

[1130, 2189, 3018, 3370, 4067, 4384, 4513, 5686, 7757, 12414]

In [ ]:
poi_df.iloc[idx,0]=true_poi

In [ ]:
poi_df.iloc[idx,0]

135      warung kuning dekat basecamp gunung andong via
143                                         sisun kel.,
169                 balilaris pie susu & lapis legit by
189                             karang sari kpri makmur
219                                  masjid al-muttaqin
                              ...                      
49837                    maje ta'lim nuhda al - mansyu,
49889                      madrasah tsanaw negeri sale,
49932                              sd neg 022iii lempur
49949                                                  
49969                                       king's gym,
Name: POI, Length: 1034, dtype: object

In [ ]:
poi_df.to_csv('poi_test_2.csv')

In [ ]:
!cp 'poi_test.csv' "/content/gdrive/My Drive/LOMBA"

cp: 'poi_test.csv' and '/content/gdrive/My Drive/LOMBA/poi_test.csv' are the same file
